In [ ]:
!pip install lightfm

from lightfm.data import Dataset
from lightfm import LightFM
from scipy.sparse import coo_matrix
from collections.abc import Iterable
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Pipeline для LightFM.

In [ ]:
class HybridRecommender:
    """
    Рекомендательная система, основанная на LightFM. Работает с implicit и explicit данными и
    только с pd.DataFrame (pandas таблица).
    """

    def __init__(self, no_components, user_id_column_name, 
                 item_id_column_name, rating_column_name=None,
                 user_features_names=None, item_features_names=None):
        """
        :param no_components: Количество компонент в LightFM.
        :param user_id_column_name: Имя столбца в pd.DataFrame с id пользователя.
        :param item_id_column_name: Имя столбца в pd.DataFrame с id item'а.
        :param rating_column_name: Имя столбца в pd.DataFrame с рэйтингами item'ов.
        :param user_features_names: Список с именами столбцов, содержащих признаки пользователей.
        :param item_features_names: Список с именами столбцов, содержащих признаки item'ов.
        """

        self.no_components = no_components
        self.user_id_column_name = user_id_column_name
        self.item_id_column_name = item_id_column_name
        self.rating_column_name = rating_column_name
        self.user_features_names = user_features_names or []
        self.item_features_names = [col for col in (item_features_names or []) if col != rating_column_name]

    def _build_features_matrix(self, data, feature_names, id_col, build_method):
        """Вспомогательный метод для создания разреженной матрицы признаков."""

        if not feature_names:
            return None
        
        features_df = data.drop_duplicates(subset=[id_col])
        feature_generator = (
            (getattr(row, id_col), [f'{col}:{getattr(row, col)}' for col in feature_names if pd.notna(getattr(row, col))])
            for row in features_df.itertuples()
        )

        return build_method(feature_generator)

    def fit(self, train_data, submission_data=None, epochs=15, num_threads=4, verbose=True):
        """
        Обучает модель на основе таблицы, где показаны взаимодействия пользователей и item'ов.
        Может также использовать признаки пользователей и item'ов.

        :param train_data: pd.DataFrame со столбцами id пользователей и id item'ов и, возможно, их признаками.
        :param submission_data: (Опционально) pd.DataFrame, который будет использоваться для рекомендаций.
                                 Полезен, чтобы LightFM "узнал" о пользователях из сабмита заранее.
        :param epochs: Количество эпох для обучения модели.
        :param num_threads: Количество потоков для распараллеливания обучения в LightFM.
        :param verbose: Выводить ли информацию об обучении.
        """

        train_data = self._preprocess_features(train_data)
        if submission_data is not None:
            submission_data = self._preprocess_features(submission_data)

        all_user_features = {f'{col}:{val}' for col in self.user_features_names for val in train_data[col].dropna().unique()}
        if submission_data is not None:
            user_features_sub = {f'{col}:{val}' for col in self.user_features_names if col in submission_data
                                 for val in submission_data[col].dropna().unique()}
            all_user_features.update(user_features_sub)

        all_item_features = {f'{col}:{val}' for col in self.item_features_names for val in train_data[col].dropna().unique()}
        all_users = pd.concat([train_data[self.user_id_column_name], submission_data[self.user_id_column_name]],
                              ignore_index=True).unique() if submission_data is not None else train_data[self.user_id_column_name].unique()

        interactions_cols = [self.user_id_column_name, self.item_id_column_name]
        if self.rating_column_name:
            interactions_cols.append(self.rating_column_name)
        interactions_data = train_data[interactions_cols].itertuples(index=False)

        self.dataset = Dataset(); self.dataset.fit(users=all_users, items=train_data[self.item_id_column_name].unique(),
                                                   user_features=all_user_features, item_features=all_item_features)
        interactions, weights = self.dataset.build_interactions(interactions_data)

        self.user_features_matrix = self._build_features_matrix(train_data, self.user_features_names, self.user_id_column_name,
                                                                self.dataset.build_user_features)
        self.item_features_matrix = self._build_features_matrix(train_data, self.item_features_names, self.item_id_column_name,
                                                                self.dataset.build_item_features)

        self.user_id_map, _, self.item_id_map, _ = self.dataset.mapping()
        self.inv_item_map = {v: k for k, v in self.item_id_map.items()}

        self.model = LightFM(loss='warp', no_components=self.no_components, random_state=42)
        self.model.fit(interactions, sample_weight=weights, user_features=self.user_features_matrix,
                       item_features=self.item_features_matrix, epochs=epochs, num_threads=num_threads, verbose=verbose)

        self.users_items_interactions_dict = train_data.groupby(self.user_id_column_name)[self.item_id_column_name].apply(set).to_dict()
        self.items_popularity = train_data[self.item_id_column_name].value_counts()
        self.sorted_items_by_popularity = self.items_popularity.index.tolist()
        self.all_item_ids = np.arange(interactions.shape[1], dtype=np.int32)

        return self

    def predict(self, df, n_recommendations, num_threads=4, return_scores=False, verbose=True):
        """
        Даёт рекомендации для пользователей.
        
        :param df: pd.DataFrame с пользователями для предсказания.
        :param n_recommendations: Количество рекомендаций для каждого пользователя.
        :param num_threads: Количество потоков для распараллеливания прогнозов в LightFM.
        :param return_scores: Если True, возвращает список кортежей (item_id, score). Иначе - только список item_id.
        :param verbose: Выводить ли информацию о процессе.
        :return: Словарь с рекомендациями {user_id: [rec_1, rec_2, ...]}.
        """

        df = self._preprocess_features(df)
        all_recommendations = {}

        known_user_mask = df[self.user_id_column_name].isin(self.user_id_map)
        known_user_ids_original = df.loc[known_user_mask, self.user_id_column_name].unique()
        unknown_user_ids_original = df.loc[~known_user_mask, self.user_id_column_name].unique()

        # обработка неизвестных пользователей
        for user_id in unknown_user_ids_original:
            top_popular = self.sorted_items_by_popularity[:n_recommendations]
            if return_scores:
                all_recommendations[user_id] = list(zip(top_popular, self.items_popularity.loc[top_popular].tolist()))
            else:
                all_recommendations[user_id] = top_popular
        
        # обработка известных пользователей
        if len(known_user_ids_original) > 0:
            n_items = len(self.all_item_ids)
            iterator = tqdm(known_user_ids_original, desc='Генерация рекомендаций') if verbose else known_user_ids_original
            
            for user_id_orig in iterator:
                user_id_int = self.user_id_map[user_id_orig]
                user_id_repeated = np.full(n_items, user_id_int, dtype=np.int32)

                scores = self.model.predict(user_id_repeated, self.all_item_ids, user_features=self.user_features_matrix,
                                            item_features=self.item_features_matrix, num_threads=num_threads)

                used_items = self.users_items_interactions_dict.get(user_id_orig, set())
                if used_items:
                    used_internal_ids = [self.item_id_map[item] for item in used_items if item in self.item_id_map]
                    scores[used_internal_ids] = -np.inf

                # остальная логика без изменений
                top_items_internal = np.argsort(-scores)[:n_recommendations]
                recs = [self.inv_item_map[i] for i in top_items_internal]

                if return_scores:
                    all_recommendations[user_id_orig] = list(zip(recs, scores[top_items_internal]))
                else:
                    all_recommendations[user_id_orig] = recs

        return all_recommendations
    
    def convert_recommendations_dict_to_pandas(self, recommendations_dict):
        """
        Конвертирует словарь с рекомендациями в pd.DataFrame (pandas таблицу).

        :param recommendations_dict: Словарь с рекомендациями: {u1: [i1, i2, ...], u2: [i1, i2, ...], ...}.
        :return: pd.DataFrame с рекомендациями.
        """

        recommendations = []
        for user_id, rec_list in recommendations_dict.items():
            row = {self.user_id_column_name: user_id}
            for i, item_id in enumerate(rec_list):
                if isinstance(item_id, tuple): # Обработка случая, когда predict возвращает (item, score)
                    item_id = item_id[0]
                row[f'{self.item_id_column_name}_{i + 1}'] = item_id
            recommendations.append(row)
        return pd.DataFrame(recommendations)

    def _preprocess_features(self, df):
        """
        Быстро и корректно стандартизирует признаки в DataFrame. Числа без дробной части приводятся к строке
        без ".0". Это нужно для того, чтобы LightFM не считал признаки "12" и "12.0" разными.
        """

        df_copy = df.copy()
        feature_cols = set(self.user_features_names + self.item_features_names) & set(df_copy.columns)
        
        for col in feature_cols:
            df_copy[col] = df_copy[col].astype(str)
            s_numeric = pd.to_numeric(df_copy[col], errors='coerce')
            is_integer_mask = pd.notna(s_numeric) & (s_numeric == np.floor(s_numeric))
            
            if is_integer_mask.any():
                df_copy.loc[is_integer_mask, col] = s_numeric[is_integer_mask].astype(int).astype(str)
                
        return df_copy

Пример использования с implicit данными:

In [ ]:
# создание данных
train_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_1', 'user_2', 'user_2', 'user_3'],
    'community_id': ['item_A', 'item_B', 'item_C', 'item_D', 'item_A']
})
submission_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_3', 'user_4']
})

model = HybridRecommender(no_components=4, user_id_column_name='customer_id', item_id_column_name='community_id')  # создание модели
model.fit(train_data, epochs=5)  # обучение модели
recommendations_dict = model.predict(submission_data, n_recommendations=3, return_scores=True)  # рекомендация item'ов пользователям

submission_df = model.convert_recommendations_dict_to_pandas(recommendations_dict)  # из словаря в pandas таблицу
submission_df.head()

Пример использования с explicit данными:

In [ ]:
# создание данных
train_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_1', 'user_2', 'user_2', 'user_3'],
    'customer_age': [12, 12, 22, 22, 17],
    'customer_city': ['Moscow', 'Moscow', 'Ekaterinburg', 'Ekaterinburg', 'Arkhangelsk'],

    'community_id': ['item_A', 'item_B', 'item_C', 'item_D', 'item_A'],
    'community_type': ['games', 'education', 'education', 'business', 'games'],
    'community_rating': [3, 4, 4, 5, 3]
})
submission_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_3', 'user_4'],
    'customer_age': [12, 17, 122.449],
    # здесь недостаёт признака 'customer_city', но HybridRecommender всё равно правильно его обработал
})

# создание и обучение модели
model = HybridRecommender(
    no_components=4,
    user_id_column_name='customer_id',
    item_id_column_name='community_id',
    rating_column_name='community_rating',
    user_features_names=['customer_age', 'customer_city'],
    item_features_names=['community_type']

)
model.fit(train_data, submission_data, epochs=5)

recommendations_dict = model.predict(submission_data, n_recommendations=3, return_scores=True)  # рекомендация item'ов пользователям

submission_df = model.convert_recommendations_dict_to_pandas(recommendations_dict)  # из словаря в pandas таблицу
submission_df.head()

# Pipeline для Two-Stage RecSys, основанная на LightFM и мощном классификаторе.

**Инициализируйте HybridRecommender выше.**

In [ ]:
from sklearn.model_selection import train_test_split

Пример использования на explicit данных.

Пример 1:

In [ ]:
class TwoStageRecommender:
    """
    Двухэтапная рекомендательная система с автоматическим созданием признаков для ранжировщика.
    """

    def __init__(self, candidate_generator, ranker, user_id_column_name,
                 item_id_column_name, ranker_feature_names=None):
        """
        :param candidate_generator: Объект-генератор кандидатов (например, HybridRecommender).
        :param ranker: Классификатор с scikit-learn API (например, CatBoostClassifier).
        :param user_id_column_name: Имя столбца с ID пользователя.
        :param item_id_column_name: Имя столбца с ID айтема.
        :param ranker_feature_names: (Опционально) Список ИСХОДНЫХ имен столбцов-признаков.
        """
        self.candidate_generator = candidate_generator
        self.ranker = ranker
        self.user_id_column_name = user_id_column_name
        self.item_id_column_name = item_id_column_name
        self.initial_ranker_features = ranker_feature_names or []
        # хранилища для данных и признаков
        self._full_feature_data = None
        self._features_cache = None
        self._user_stats_df = None
        self._item_stats_df = None
        self._final_ranker_features = []

    def fit(self, train_data, submission_data=None, ranker_train_size=0.5,
            n_candidates_for_ranker=100, ranker_validation_split_size=0.2, ranker_fit_batch_size=16,
            generator_fit_params=None, ranker_fit_params=None):
        """
        Обучает весь пайплайн: сначала генератор кандидатов, затем ранжировщик.
        Устраняет утечку данных, разделяя train_data на две части.

        :param train_data: pd.DataFrame с обучающими взаимодействиями.
        :param submission_data: (Опционально) pd.DataFrame с пользователями для предсказания.
        :param ranker_train_size: Доля данных для обучения ранжировщика (от 0 до 1).
        :param n_candidates_for_ranker: Сколько кандидатов генерировать для обучения ранжировщика.
        :param ranker_validation_split_size: Доля данных для валидации ранжировщика.
        :param ranker_fit_batch_size: Размер батча для обучения ранжировщика.
        :param generator_fit_params: Словарь с параметрами для .fit() генератора.
        :param ranker_fit_params: Словарь с параметрами для .fit() ранжировщика.
        """
        generator_fit_params = generator_fit_params or {}
        ranker_fit_params = ranker_fit_params or {}
        
        # сохраняем все данные, содержащие признаки
        data_parts = [train_data]
        if submission_data is not None:
            data_parts.append(submission_data)
        self._full_feature_data = pd.concat(data_parts, ignore_index=True)

        # разделяем данные для устранения утечки
        generator_train_data, ranker_train_data = train_test_split(
            train_data, test_size=ranker_train_size, random_state=42
        )
        print(f"Данные разделены: {len(generator_train_data)} строк для генератора, {len(ranker_train_data)} для ранжировщика.")

        # --- Этап 1: Обучение генератора кандидатов ---
        print("\n--- Этап 1: Обучение генератора кандидатов ---")
        self.candidate_generator.fit(generator_train_data, submission_data, **generator_fit_params)

        # --- Этап 2: Обучение ранжировщика ---
        print("\n--- Этап 2: Обучение ранжировщика ---")
        self._fit_ranker(ranker_train_data, n_candidates_for_ranker, ranker_validation_split_size,
                         ranker_fit_params, batch_size=ranker_fit_batch_size)
        
        return self

    def _fit_ranker(self, ranker_train_data, n_candidates, val_size, fit_params, batch_size=256):
        """Внутренний метод для подготовки данных и обучения ранжировщика."""

        # генерация статистических признаков (без изменений)
        self._item_stats_df = ranker_train_data.groupby(self.item_id_column_name).agg(item_popularity=(
            self.user_id_column_name, 'count'), item_n_unique_users=(self.user_id_column_name, 'nunique')).reset_index()
        self._user_stats_df = ranker_train_data.groupby(self.user_id_column_name).agg(user_activity=(
            self.item_id_column_name, 'count'), user_n_unique_items=(self.item_id_column_name, 'nunique')).reset_index()
        generated = ['item_popularity', 'item_n_unique_users', 'user_activity', 'user_n_unique_items', 'generator_score']
        self._final_ranker_features = self.initial_ranker_features + generated

        all_users_for_ranker = ranker_train_data[self.user_id_column_name].unique()
        positives_set = set(zip(ranker_train_data[self.user_id_column_name], ranker_train_data[self.item_id_column_name]))

        dataset_batches = []
        # итерация по батчам пользователей
        for i in tqdm(range(0, len(all_users_for_ranker), batch_size), desc="Создание датасета для ранжировщика"):
            user_batch = all_users_for_ranker[i : i + batch_size]
            users_df_batch = pd.DataFrame(user_batch, columns=[self.user_id_column_name])

            # генерация кандидатов только для батча
            candidates_dict_batch = self.candidate_generator.predict(users_df_batch, n_recommendations=n_candidates,
                                                                     return_scores=True, verbose=False)
            if not any(candidates_dict_batch.values()): continue

            # создание временного списка с данными для DataFrame
            dataset_list_batch = [{
                self.user_id_column_name: uid, self.item_id_column_name: iid, 'generator_score': score,
                'target': 1 if (uid, iid) in positives_set else 0
            } for uid, items in candidates_dict_batch.items() for iid, score in items]

            if dataset_list_batch: dataset_batches.append(pd.DataFrame(dataset_list_batch))

        # сборка итогового датасета из всех батчей
        ranker_dataset = pd.concat(dataset_batches, ignore_index=True)
        del dataset_batches # освобождаем память

        # разделение и обучение (логика остается прежней)
        train_df, val_df = train_test_split(ranker_dataset, test_size=val_size, random_state=42, stratify=ranker_dataset['target'])
        print("Обогащение данных признаками и обучение модели...")
        x_train, y_train = self._prepare_ranker_features(train_df), train_df['target']
        x_val, y_val = self._prepare_ranker_features(val_df), val_df['target']

        if 'eval_set' not in fit_params:
            if 'catboost' in self.ranker.__class__.__name__.lower():
                fit_params['eval_set'] = (x_val, y_val) 
            else:
                fit_params['eval_set'] = [(x_val, y_val)]

        self.ranker.fit(x_train, y_train, **fit_params)
    
    def predict(self, df, n_candidates, n_recommendations, batch_size=256, verbose=True):
        """
        Генерирует финальные рекомендации для пользователей в два этапа.

        :param df: pd.DataFrame с пользователями для предсказания.
        :param n_candidates: Количество кандидатов для генерации на 1-м этапе.
        :param n_recommendations: Итоговое количество рекомендаций.
        :param batch_size: Размер батча.
        :param verbose: Выводить ли информацию о процессе.
        :return: Словарь с финальными рекомендациями {user_id: [rec_1, rec_2, ...]}.
        """

        all_users = df[self.user_id_column_name].unique()
        final_recs_dict = {}

        # итерация по батчам пользователей
        for i in tqdm(range(0, len(all_users), batch_size), desc="Финальные рекомендации"):
            user_batch = all_users[i : i + batch_size]
            df_batch = pd.DataFrame(user_batch, columns=[self.user_id_column_name])

            # генерация кандидатов для батча
            candidates_dict = self.candidate_generator.predict(df_batch, n_recommendations=n_candidates, return_scores=True, verbose=False)
            if not any(candidates_dict.values()):
                for user_id in user_batch: final_recs_dict[user_id] = []
                continue

            # создание DataFrame из кандидатов батча
            candidates_list = [{self.user_id_column_name: uid, self.item_id_column_name: iid, 'generator_score': score}
                               for uid, items in candidates_dict.items() for iid, score in items]
            candidates_df = pd.DataFrame(candidates_list)

            # предсказания и ранжирование для батча
            X_pred = self._prepare_ranker_features(candidates_df)
            candidates_df['ranker_score'] = self.ranker.predict_proba(X_pred)[:, 1]
            candidates_df = candidates_df.sort_values('ranker_score', ascending=False)
            top_ranked_df = candidates_df.groupby(self.user_id_column_name).head(n_recommendations)

            # обновление итогового словаря рекомендаций
            batch_recs_dict = top_ranked_df.groupby(self.user_id_column_name)[self.item_id_column_name].apply(list).to_dict()
            final_recs_dict.update(batch_recs_dict)

        # убеждаемся, что все пользователи из исходного запроса есть в ответе
        return {user_id: final_recs_dict.get(user_id, []) for user_id in df[self.user_id_column_name].unique()}

    def _prepare_ranker_features(self, df):
        """Внутренний метод для обогащения датафрейма признаками перед ранжированием."""

        df_with_features = df.copy()
        
        # присоединяем исходные признаки, если они были указаны
        if self.initial_ranker_features:
            if self._features_cache is None:
                # кэшируем признаки, чтобы не делать это много раз
                cols_to_cache = [self.user_id_column_name] + [c for c in self.initial_ranker_features if c in self._full_feature_data.columns]
                self._features_cache = self._full_feature_data[list(set(cols_to_cache))].drop_duplicates(subset=[self.user_id_column_name])
            
            df_with_features = df_with_features.merge(self._features_cache, on=self.user_id_column_name, how='left')

        # присоединяем статистические признаки
        df_with_features = df_with_features.merge(self._user_stats_df, on=self.user_id_column_name, how='left')
        df_with_features = df_with_features.merge(self._item_stats_df, on=self.item_id_column_name, how='left')
        
        # выбираем финальный набор признаков
        final_cols = [col for col in self._final_ranker_features if col in df_with_features.columns]
        df_final = df_with_features[final_cols].copy()
        
        # обрабатываем пропуски в зависимости от типа колонки
        for col in df_final.columns:
            if pd.api.types.is_numeric_dtype(df_final[col]):
                df_final[col] = df_final[col].fillna(0)
            else:
                df_final[col] = df_final[col].fillna('[MISSING]')
        
        return df_final
    
    def convert_recommendations_dict_to_pandas(self, recommendations_dict):
        """
        Конвертирует словарь с рекомендациями в pd.DataFrame (pandas таблицу).

        :param recommendations_dict: Словарь с рекомендациями: {u1: [i1, i2, ...], u2: [i1, i2, ...], ...}.
        :return: pd.DataFrame с рекомендациями.
        """

        return self.candidate_generator.convert_recommendations_dict_to_pandas(recommendations_dict)

In [ ]:
# создание данных
train_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_1', 'user_2', 'user_2', 'user_3', 'user_1', 'user_3', 'user_4', 'user_4', 'user_2', 'user_3'],
    'customer_age': [25, 25, 30, 30, 22, 25, 22, 45, 45, 30, 22],
    'customer_sex': ['M', 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'F', 'F', 'M'],
    'community_id': ['item_A', 'item_B', 'item_C', 'item_D', 'item_A', 'item_E', 'item_C', 'item_B', 'item_D', 'item_F', 'item_F'],
    'community_type': ['games', 'edu', 'edu', 'business', 'games', 'travel', 'edu', 'edu', 'business', 'music', 'music'],
})
submission_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_2', 'user_5'],
    'customer_age': [25, 30, 50],
    'customer_sex': ['M', 'F', 'M'],
})

# создание компонентов
candidate_generator = HybridRecommender(
    no_components=15,
    user_id_column_name='customer_id',
    item_id_column_name='community_id'
    
)
ranker = CatBoostClassifier(
    n_estimators=1000,
    depth=4,
    learning_rate=0.03,
    auto_class_weights='Balanced',
    eval_metric='AUC',
    early_stopping_rounds=300,
    random_state=42,
    verbose=100
)

# создание двухэтапной модели
pipeline = TwoStageRecommender(
    candidate_generator=candidate_generator,
    ranker=ranker,
    user_id_column_name='customer_id',
    item_id_column_name='community_id'
)

# обучение генератора кандидатов
pipeline.fit(
    train_data=train_data,  # train_data
    submission_data=submission_data,
    ranker_train_size=0.5,
    n_candidates_for_ranker=100,
    ranker_validation_split_size=0.2,
    generator_fit_params={'epochs': 15, 'verbose': True},
    ranker_fit_params=None
)

# получение рекомендаций для сабмита
recommendations_dict = pipeline.predict(
    df=submission_data,
    n_candidates=100,
    n_recommendations=7,
)

Пример 2:

In [ ]:
# создание данных
train_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_1', 'user_2', 'user_2', 'user_3', 'user_1', 'user_3', 'user_4', 'user_4', 'user_2', 'user_3'],
    'customer_age': [25, 25, 30, 30, 22, 25, 22, 45, 45, 30, 22],
    'customer_sex': ['M', 'M', 'F', 'F', 'M', 'M', 'M', 'F', 'F', 'F', 'M'],
    'community_id': ['item_A', 'item_B', 'item_C', 'item_D', 'item_A', 'item_E', 'item_C', 'item_B', 'item_D', 'item_F', 'item_F'],
    'community_type': ['games', 'edu', 'edu', 'business', 'games', 'travel', 'edu', 'edu', 'business', 'music', 'music'],
})
submission_data = pd.DataFrame({
    'customer_id': ['user_1', 'user_2', 'user_5'],
    'customer_age': [25, 30, 50],
    'customer_sex': ['M', 'F', 'M'],
})

# создание компонентов
candidate_generator = HybridRecommender(
    no_components=15,
    user_id_column_name='customer_id',
    item_id_column_name='community_id',
    user_features_names=['customer_age', 'customer_sex'],
    item_features_names=['community_type']
)
ranker = CatBoostClassifier(
    n_estimators=1000,
    depth=4,
    learning_rate=0.03,
    auto_class_weights='Balanced',
    cat_features=['customer_sex', 'community_type'],
    eval_metric='AUC',
    early_stopping_rounds=300,
    random_state=42,
    verbose=100
)
ranker_features = ['customer_age', 'customer_sex', 'community_type']

# создание двухэтапной модели
pipeline = TwoStageRecommender(
    candidate_generator=candidate_generator,
    ranker=ranker,
    user_id_column_name='customer_id',
    item_id_column_name='community_id',
    ranker_feature_names=ranker_features
)

# обучение генератора кандидатов
pipeline.fit(
    train_data=train_data,  # train_data
    submission_data=submission_data,
    ranker_train_size=0.5,
    n_candidates_for_ranker=100,
    ranker_validation_split_size=0.2,
    generator_fit_params={'epochs': 15, 'verbose': True},
    ranker_fit_params=None
)

# получение рекомендаций для сабмита
recommendations_dict = pipeline.predict(
    df=submission_data,
    n_candidates=100,
    n_recommendations=7,
)